In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy import linalg
from datetime import datetime, timedelta
import scipy.optimize as optimization
from scipy.integrate import odeint
from scipy.stats import poisson
from ddeint import ddeint

In [2]:
# Please see "data-tailor-COVID19-Italy" on how to generate the italydata.csv file.

data=pd.read_csv("./italydata.csv",low_memory=False)
dataframe=pd.DataFrame(data)

In [3]:
# Please see "data-fitting-COVID19-Italy" on how to generate the inflow.pickle file.

import pickle
inflowlist = pickle.load( open( "inflow.pickle", "rb" ) )

In [4]:
# Set-up timestep.

DDEtime=np.linspace(0,409,40901)

# Model parameters.

PC0=26
PM0=103
R0=1
gammapc=0.1
gammapm=0.125
mupm=0
eta=1
K=10000
pd=0.1
alpha=-math.log(1-pd)
pc=0.1
xi=-math.log(1-pc)
omega=1.5*alpha
tau=14
lamda=1/40
cumdeath=72950
epsilon=0.1
sigma=1/14.
f=0.1
gammat=1/(1/gammapc-1)
mut=f*gammat/(1-f)

In [5]:
# Time-dependent parameters.

# Stage-dependent progression rate p:

def pfunc(t):
    if 0<=t<=209:
        return 0.0378
    if t>209:
        return 0.0038
    
# Stage-dependent death rate of critical cases mu_P^c:

def mupcfunc(t):
    if 0<=t<=209:
        return 0.2050
    if t>209:
        return 0.1457
    
# Inflow rate of newly identified cases \kappa I(t):
    
def inflowfunc(t):
    if t<0:
        return 0
    if t>=0:
        return inflowlist[np.floor(t).astype(int)]

In [6]:
# Initial condition for the first-time donor population with delay:

def D1func(t):
    return 0

In [7]:
# Set the starting day of large-scale use of convalescent blood transfusion.

startday = 37

# Time-dependent rate of donor screening:
    
def epsilonfunc(t):
    if 0<=t<=startday:
        return 0.
    if t>startday:
        return epsilon
    
# Time-dependent loss rate of recovered individuals:
    
def sigmafunc(t):
    if 0<=t<=startday:
        return xi
    if t>startday:
        return sigma
    
# Treatment-donation-stockpile system:

def DDEmodel(Y,t):
    PC,PM,TT,RR,D1,D0,DM,BB=Y(t)
    PCd,PMd,TTd,RRd,D1d,D0d,DMd,BBd=Y(t-tau)
    return np.array([pfunc(t)*PM-(mupcfunc(t)+gammapc)*PC-eta*BB*PC/(BB+K), inflowfunc(t)-(mupm+gammapm+pfunc(t))*PM, 
                    eta*BB*PC/(BB+K)-(mut+gammat)*TT, gammapm*PM-sigmafunc(t)*RR,
                    epsilonfunc(t)*sigmafunc(t)*RR-(alpha+xi)*D1, alpha*(D1+DM)-xi*D0-alpha*math.exp(-xi*tau)*(D1d+DMd),
                    alpha*math.exp(-xi*tau)*(D1d+DMd)-(alpha+xi)*DM, omega*(D1+DM)-eta*BB*PC/(BB+K)-lamda*BB])

# Initial condition:

g = lambda t : np.array([PC0,PM0,0, R0, D1func(t),0,0,0])

# Solution:

DDEsolday37=ddeint(DDEmodel,g,DDEtime)

In [20]:
# Compute the reduced deaths and maximal demand of apheresis under different starting days.

deathlist=[]
maxlist=[]

for i in timelist:
    startday = i
    def epsilonfunc(t):
        if 0<=t<=startday:
            return 0.
        if t>startday:
            return epsilon
    
    def sigmafunc(t):
        if 0<=t<=startday:
            return xi
        if t>startday:
            return sigma
        
    def DDEmodel(Y,t):
        PC,PM,TT,RR,D1,D0,DM,BB,DD=Y(t)
        PCd,PMd,TTd,RRd,D1d,D0d,DMd,BBd,DDd=Y(t-tau)
        return np.array([pfunc(t)*PM-(mupcfunc(t)+gammapc)*PC-eta*BB*PC/(BB+K), inflowfunc(t)-(mupm+gammapm+pfunc(t))*PM, 
                    eta*BB*PC/(BB+K)-(mut+gammat)*TT, gammapm*PM-sigmafunc(t)*RR,
                    epsilonfunc(t)*sigmafunc(t)*RR-(alpha+xi)*D1, alpha*(D1+DM)-xi*D0-alpha*math.exp(-xi*tau)*(D1d+DMd),
                    alpha*math.exp(-xi*tau)*(D1d+DMd)-(alpha+xi)*DM, omega*(D1+DM)-eta*BB*PC/(BB+K)-lamda*BB,
                    mupcfunc(t)*PC+mupm*PM+mut*TT])
    g = lambda t : np.array([PC0,PM0,0, R0, D1func(t),0,0,0,7])
    DDEsol=ddeint(DDEmodel,g,DDEtime)
    deathlist.append(cumdeath-DDEsol[-1,8])
    maxlist.append(max(DDEsol[:,4]+DDEsol[:,6])*omega)